In [2]:
import random
import pandas as pd

In [5]:
def vlox_crossover(parent1, parent2):
    # Get lengths of the parents
    size1, size2 = len(parent1), len(parent2)
    
    # Determine crossover points for both parents
    p1_start, p1_end = sorted(random.sample(range(size1), 2))
    p2_start, p2_end = sorted(random.sample(range(size2), 2))
    
    # Create the child initialized to None
    child = [None] * max(size1, size2)
    
    # Copy the segment from the first parent to the child
    child[:p1_end - p1_start] = parent1[p1_start:p1_end]
    
    # Add the segment from the second parent into the child, preserving order and avoiding duplicates
    remaining_positions = [i for i, x in enumerate(child) if x is None]
    segment2 = parent2[p2_start:p2_end]
    remaining_segment2 = [x for x in segment2 if x not in child]
    
    for i, location in zip(remaining_positions, remaining_segment2):
        child[i] = location
    
    # Fill the remaining positions with the rest of parent2, preserving order and avoiding duplicates
    current_index = remaining_positions[len(remaining_segment2):]
    for location in parent2:
        if location not in child and current_index:
            child[current_index.pop(0)] = location
    
    # Trim any None values for the final valid child tour
    child = [location for location in child if location is not None]
    
    return child

# Example usage
parent1 = ['A', 'B', 'C', 'D', 'E']
parent2 = ['B', 'D', 'A', 'E', 'C', 'F']

child = vlox_crossover(parent1, parent2)
print("Child:", child)


Child: ['B', 'D', 'A', 'E', 'C', 'F']


In [6]:
def swap_mutation(tour, mutation_rate=0.1):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(tour)), 2)
        tour[idx1], tour[idx2] = tour[idx2], tour[idx1]
    return tour

def insertion_mutation(tour, mutation_rate=0.1):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(tour)), 2)
        location = tour.pop(idx1)
        # Ensure no duplicates and all locations are included
        if location not in tour:
            tour.insert(idx2, location)
    return tour

def scramble_mutation(tour, mutation_rate=0.1):
    if random.random() < mutation_rate:
        start, end = sorted(random.sample(range(len(tour)), 2))
        sublist = tour[start:end]
        random.shuffle(sublist)
        tour[start:end] = sublist
    return tour

def inversion_mutation(tour, mutation_rate=0.1):
    if random.random() < mutation_rate:
        start, end = sorted(random.sample(range(len(tour)), 2))
        tour[start:end] = reversed(tour[start:end])
    return tour

In [7]:
def crossover_and_mutate(parent1, parent2, mutation_rate=0.1):
    # Perform crossover to create a child
    child = vlox_crossover(parent1, parent2)
    print(child)
    
    # List of available mutation functions
    mutations = [swap_mutation, insertion_mutation, scramble_mutation, inversion_mutation]
    
    # Randomly select one mutation function
    mutation_function = random.choice(mutations)
    
    # Apply the selected mutation function to the child
    mutated_child = mutation_function(child, mutation_rate)
    
    # Ensure no duplicates and all locations are included
    missing_locations = set(parent1) - set(mutated_child)
    duplicate_locations = set([loc for loc in mutated_child if mutated_child.count(loc) > 1])
    
    for duplicate in duplicate_locations:
        for missing in missing_locations:
            
            idx = mutated_child.index(duplicate)
            mutated_child[idx] = missing
            break
    
    return mutated_child

# Example Usage
parent1 = ['A', 'B', 'C', 'D', 'E']
parent2 = ['B', 'D', 'A', 'E', 'C', 'F']

# Generate a child with crossover and mutation
child = crossover_and_mutate(parent1, parent2, mutation_rate=0.2)
print("Child after crossover and mutation:", child)

['A', 'B', 'C', 'D', 'E', 'F']
Child after crossover and mutation: ['A', 'B', 'C', 'D', 'E', 'F']


In [ ]:
def crossover_and_mutate(parent1, parent2, mutation_rate=0.1):
    # Perform crossover to create a child
    child = vlox_crossover(parent1, parent2)
    print(child)
    
    # List of available mutation functions
    mutations = [swap_mutation, insertion_mutation, scramble_mutation, inversion_mutation]
    
    # Randomly select one mutation function
    mutation_function = random.choice(mutations)
    
    # Apply the selected mutation function to the child
    mutated_child = mutation_function(child, mutation_rate)
    
    # Ensure no duplicates and all locations are included
    missing_locations = set(parent1) - set(mutated_child)
    duplicate_locations = set([loc for loc in mutated_child if mutated_child.count(loc) > 1])
    
    for duplicate in duplicate_locations:
        for missing in missing_locations:
            
            idx = mutated_child.index(duplicate)
            mutated_child[idx] = missing
            break
    
    return mutated_child

# Example Usage
parent1 = ['A', 'B', 'C', 'D', 'E']
parent2 = ['B', 'D', 'A', 'E', 'C', 'F']

# Generate a child with crossover and mutation
child = crossover_and_mutate(parent1, parent2, mutation_rate=0.2)
print("Child after crossover and mutation:", child)

In [8]:
def generate_initial_population(num_tours, min_locations, max_locations):
    """
    Generate an initial population of tours with varying numbers of locations.

    Parameters:
    num_tours (int): The number of tours (individuals) in the population.
    min_locations (int): The minimum number of locations in any tour.
    max_locations (int): The maximum number of locations in any tour.

    Returns:
    list of lists: A list containing the initial population of tours.
    """
    population = []
    
    for _ in range(num_tours):
        # Randomly determine the number of locations for the current tour
        num_locations = random.randint(min_locations, max_locations)
        
        # Generate a list of location IDs
        locations = list(range(num_locations))
        
        # Shuffle the list to create a random permutation
        random.shuffle(locations)
        
        # Add the generated tour to the population
        population.append(locations)
    
    return population

# Example Usage
num_tours = 10  # Number of tours in the initial population
min_locations = 3  # Minimum number of locations in any tour
max_locations = 7  # Maximum number of locations in any tour

# Generate the initial population
initial_population = generate_initial_population(num_tours, min_locations, max_locations)
print("Initial Population:")
for i, tour in enumerate(initial_population):
    print(f"Tour {i + 1}: {tour}")

Initial Population:
Tour 1: [1, 0, 3, 4, 2, 5]
Tour 2: [2, 5, 0, 4, 3, 1]
Tour 3: [2, 3, 0, 1, 5, 4]
Tour 4: [2, 0, 3, 5, 1, 4]
Tour 5: [2, 3, 4, 0, 1]
Tour 6: [0, 2, 1]
Tour 7: [0, 2, 6, 5, 3, 1, 4]
Tour 8: [4, 0, 3, 6, 1, 2, 5]
Tour 9: [3, 1, 6, 0, 4, 2, 5]
Tour 10: [2, 3, 0, 1]
